In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import csv
import dask.dataframe as dd
from dateutil import parser
import pickle
import processData

Using TensorFlow backend.


In [ ]:
import sys
sys.path.insert(0, "../")
sys.path.insert(0, "../scripts/")

In [3]:
def try_parse(text):
    try:
        parsed = parser.parse(text)
        
        return parsed

    except:

        return "nan"

In [4]:
def createTimeDict():
    TIME_PATH = "./time/*.tsv"
    TIME_SAVE = "id_date.pickle"
    
    df = dd.read_csv(TIME_PATH, delimiter='\t', error_bad_lines=False, header=None, names=['ID', 'Date'],
                     encoding='utf8', quoting=csv.QUOTE_NONE)
    pdf = df.compute(error_bad_lines=False, warn_bad_lines=True)
    pdf['Date'] = pdf['Date'].apply(try_parse)
    pdf_dict = pdf.set_index('ID').to_dict()['Date']
    
    with open(TIME_SAVE, 'wb') as savefile:
        pickle.dump(pdf_dict, savefile, pickle.HIGHEST_PROTOCOL)

In [52]:
def createCombined(id_ht, id_ht_pred, id_datetime, id_phone):
    ID_INDEX = 0
    LONG_INDEX = -1
    LAT_INDEX = -2
    STATE_INDEX = -5
    CITY_INDEX = -6

    IN_PATH = "2016_2017_df.csv"
    OUT_PATH = "2016_2017_combined.csv"

    with open(IN_PATH, 'r') as datafile, open(OUT_PATH, 'w') as outfile:
        reader = csv.reader(datafile, delimiter=',')
        next(reader)

        writer = csv.writer(outfile, delimiter=',')
        outline = ["doc", "phone", "ht", "ht_pred", "year", "datetime", "month", "day", "city", "state", "lat", "long"]
        writer.writerow(outline)
            
        for line in reader:                
            doc = line[ID_INDEX].strip()
            long = line[LONG_INDEX].strip()
            lat = line[LAT_INDEX].strip()
            state = line[STATE_INDEX].strip()
            city = line[CITY_INDEX].strip()
            
            ht = id_ht.get(doc, "nan")
            ht_pred = id_ht_pred.get(doc, "nan")
            datetime = id_datetime.get(doc, "nan")
            phone = id_phone.get(doc, "nan")

            if datetime == "nan":
                year = "nan"
                month = "nan"
                day = "nan"
            else:
                year = datetime.year
                month = datetime.month
                day = datetime.day

            outline = [doc, phone, ht, ht_pred, datetime, year, month, day, city, state, lat, long]

            writer.writerow(outline)

In [6]:
#id_ht = processData.createDedupe(DEDUPE_PATHS = ["2016_predictions.csv", "2017_predictions.csv"],
#                                 ID_INDEX=0, DEDUPE_INDEX=1, DEDUPE_SAVE = "2016_2017_id_ht.pickle")

In [7]:
#id_ht_pred = processData.createDedupe(DEDUPE_PATHS = ["2016_predictions.csv", "2017_predictions.csv"],
#                                      ID_INDEX=0, DEDUPE_INDEX=2, DEDUPE_SAVE = "2016_2017_id_ht_pred.pickle")

In [8]:
#createTimeDict()

In [9]:
id_ht = processData.getDedupe(PATH = "2016_2017_id_ht.pickle")

In [10]:
id_ht_pred = processData.getDedupe(PATH = "2016_2017_id_ht_pred.pickle")

In [11]:
id_datetime = processData.getDedupe(PATH = "2016_2017_id_date.pickle")

In [12]:
id_phone = processData.getDedupe(PATH = "2016_2017_dedupe.pickle")

In [53]:
createCombined(id_ht, id_ht_pred, id_datetime, id_phone)